In [1]:
from models.cnn import CNN
import os
import torch
import torch.nn as nn
from torch.optim import Adam, lr_scheduler
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("cuda")
root = "./datasets"
download = os.path.exists("./datasets/MNIST")

model = CNN()
model.to(device)

trainset = datasets.MNIST(root="./datasets", download=download, train=True, transform=transforms.ToTensor())
testset = datasets.MNIST(root="./datasets", download=download, train=False, transform=transforms.ToTensor())
trainloader = DataLoader(trainset, shuffle=True, batch_size=1024)
testloader = DataLoader(testset, shuffle=False, batch_size=128)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=1e-3)

epochs = 100
best_val_acc = 0
for e in range(epochs):
    with tqdm(trainloader,desc=f"{e+1}/{epochs} epochs") as t:
        running_correct = 0
        running_loss = 0
        running_total = 0
        model.train()
        for i, (x,y) in enumerate(t):
            out = model(x.to(device))
            loss = loss_fn(out,y.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_correct += (out.max(dim=1)[1]==y.to(device)).sum().item()
            running_loss += loss.item()*x.size(0)
            running_total += x.size(0)
            if i < len(trainloader)-1:
                t.set_postfix({
                    "train_acc": running_correct/running_total,
                    "train_loss": running_loss/running_total
                })
            else:
                model.eval()
                val_correct = 0
                val_loss = 0
                val_total = 0
                with torch.no_grad():
                    for _, (x,y) in zip(range(4),testloader):
                        out = model(x.to(device))
                        loss = loss_fn(out,y.to(device))
                        val_correct += (out.max(dim=1)[1]==y.to(device)).sum().item()
                        val_loss += loss.item()*x.size(0)
                        val_total += x.size(0)
                        t.set_postfix({
                            "train_acc": running_correct/running_total,
                            "train_loss": running_loss/running_total,
                            "val_acc": val_correct/val_total,
                            "val_loss": val_loss/val_total
                        })
                    if val_correct/val_total >= best_val_acc:
                        best_val_acc = val_correct/val_total
                        torch.save(model.state_dict(), "./model_weights/mnist_cnn_clean.pt")

62/100 epochs: 100%|██████████| 59/59 [00:04<00:00, 12.61it/s, train_acc=0.994, train_loss=0.0166, val_acc=0.994, val_loss=0.0187]


100/100 epochs: 100%|██████████| 59/59 [00:04<00:00, 12.64it/s, train_acc=0.996, train_loss=0.0127, val_acc=1, val_loss=0.00474] 
